In [3]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Загрузка токенизатора и модели
import torch
from transformers import AutoTokenizer, AutoModel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained("distilbert/distilbert-base-uncased", torch_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from torch.utils.data import Dataset
import re
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = load_dataset("ashraq/movielens_ratings", split="train")

n = 10000
small_dataset = dataset.select(range(n))
train_dataset = small_dataset.select(range(int(0.8 * n)))
eval_dataset = small_dataset.select(range(int(0.8 * n), n))


from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from torch.utils.data import Dataset


dataset = load_dataset("ashraq/movielens_ratings", split="train")

n = 1000
indices = list(range(n))
train_indices = indices[:int(0.8 * n)]
eval_indices = indices[int(0.8 * n):]

small_dataset = dataset.select(indices)
train_dataset = small_dataset.select(train_indices)
eval_dataset = small_dataset.select(eval_indices)


class MovieLensHFDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.pairs = [(i, j) for i in range(len(dataset)) for j in range(len(dataset))]


    def __getitem__(self, idx):
        idx1, idx2 = self.pairs[idx]
        row1 = self.dataset[idx1]
        row2 = self.dataset[idx2]

        user_id1 = row1["user_id"]
        movie_title1 = row1["title"]
        genres1 = row1["genres"]
        rating1 = row1["rating"]

        user_id2 = row2["user_id"]
        movie_title2 = row2["title"]
        genres2 = row2["genres"]
        rating2 = row2["rating"]

        prompt1 = (
            f"Movie 1\n"
            f"User ID: {user_id1}\n"
            f"Movie title: {movie_title1}\n"
            f"Genres: {genres1}\n"
        )

        prompt2 = (
            "Movie 2\n"
            f"User ID: {user_id2}\n"
            f"Movie title: {movie_title2}\n"
            f"Genres: {genres2}\n"
        )

        target = 1 if rating1 >= rating2 else -1

        inputs1 = self.tokenizer(
            prompt1,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt",
        ).to(device)

        inputs2 = self.tokenizer(
            prompt2,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt",
        ).to(device)

        x1 = {key: val.squeeze(0) for key, val in inputs1.items() if isinstance(key, (int, float, str))}
        x2 = {key: val.squeeze(0) for key, val in inputs2.items() if isinstance(key, (int, float, str))}


        return x1, x2, torch.tensor(target).to(device)


    def __len__(self):
        # assuming all possible pairs could be generated
        return len(self.dataset) * len(self.dataset)


train = MovieLensHFDataset(train_dataset, tokenizer)
val = MovieLensHFDataset(eval_dataset, tokenizer)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)




dataset_infos.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

(…)-00000-of-00001-8c8c7645a52d95e5.parquet:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

(…)-00000-of-00001-609ec132d91847f9.parquet:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/891382 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/99043 [00:00<?, ? examples/s]

In [6]:
import wandb


wandb.init(
    # set the wandb project where this run will be logged
    project="NLP project",
    reinit=True,
    # track hyperparameters and run metadata,
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
import torch
import torch.nn as nn

class RankingModel(nn.Module):
    def __init__(self, hidden_size=768, num_classes=1):
        super(RankingModel, self).__init__()
        self.dense = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.dense(x)


rank_model = RankingModel().to(device)
criterion = nn.MarginRankingLoss()

In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.autograd import Variable
from transformers import Trainer, BertModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm


class RankingTrainer:
    def __init__(self, base_model, rank_model, criterion, optimizer_cls=Adam, lr=0.00001):
        self.base_model = base_model
        self.rank_model = rank_model
        self.criterion = criterion
        self.optimizer = optimizer_cls(self.rank_model.parameters(), lr=lr)

    def train(self, train_dataset, validation_dataset, epochs, batch_size):
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
        self.rank_model.train()
        for epoch in range(epochs):
            total_loss = 0
            for x1, x2, y_true in tqdm(train_dataloader):
                self.optimizer.zero_grad()
                with torch.no_grad():
                    output1, output2 = self.base_model(**x1), self.base_model(**x2)
                y1, y2 = output1.last_hidden_state[:, 0, :].to(torch.float32), output2.last_hidden_state[:, 0, :].to(torch.float32)
                y1, y2 = self.rank_model(y1)[:, 0], self.rank_model(y2)[:, 0]
                loss = self.criterion(y1, y2, y_true)
                loss.backward()
                total_loss += loss.item()
                self.optimizer.step()
            avg_loss = total_loss / len(train_dataloader)
            print(f'Training loss at epoch {epoch}: {avg_loss}')
            wandb.log({
                "epoch": "epoch",
                "train_loss": avg_loss,
            })
            self.evaluate(validation_dataloader)


    def evaluate(self, dataloader):
        self.rank_model.eval()
        total_loss = 0
        pred_scores = []
        true_scores = []
        with torch.no_grad():
            for x1, x2, y_true in tqdm(dataloader):
                output1, output2 = self.base_model(**x1), self.base_model(**x2)
                y1, y2 = output1.last_hidden_state[:, 0, :].to(torch.float32), output2.last_hidden_state[:, 0, :].to(torch.float32)
                y1, y2 = self.rank_model(y1)[:, 0], self.rank_model(y2)[:, 0]
                pred_scores.extend((y1-y2).cpu().numpy())
                true_scores.extend(y_true.cpu().numpy())

        pred_labels = [1 if score > 0 else -1 for score in pred_scores]

        accuracy = accuracy_score(true_scores, pred_labels)
        precision = precision_score(true_scores, pred_labels)
        recall = recall_score(true_scores, pred_labels)
        f1 = f1_score(true_scores, pred_labels)
        roc_auc = roc_auc_score(true_scores, pred_scores)
        print(f'Evaluation Results - Accuracy: {accuracy}, Precision: {precision}, Recall:{recall}, F1: {f1}, ROC-AUC: {roc_auc}')

        wandb.log({
            "val_accuracy": accuracy,
            "val_precision": precision,
            "val_recall": recall,
            "val_f1": f1,
            "val_roc_auc": roc_auc
        })




rank_trainer = RankingTrainer(model, rank_model, criterion)
rank_trainer.train(train, val, epochs=2, batch_size=256)

 72%|███████▏  | 1810/2500 [20:05<07:46,  1.48it/s]